### Multithreading

La programación multihilo (multithreading) es una técnica fundamental en el desarrollo de software moderno que permite a un programa ejecutar múltiples operaciones concurrentes. Esta capacidad es crucial para mejorar el rendimiento y la eficiencia de las aplicaciones, especialmente en sistemas con múltiples núcleos de procesamiento. A continuación, se examinan diversos aspectos de la programación multihilo, incluyendo la creación y gestión de hilos, la sincronización de hilos, problemas clásicos como deadlocks, livelocks y condiciones de carrera, y algoritmos de sincronización como el algoritmo de Lamport y el algoritmo de Ricart-Agrawala.



**Creación y gestión de hilos**

En la programación multihilo, un hilo (thread) es la unidad más pequeña de procesamiento que puede ser gestionada de manera independiente por un sistema operativo. Los hilos son utilizados para realizar tareas concurrentes dentro de un proceso, compartiendo el mismo espacio de direcciones y recursos.

La creación de hilos es el proceso mediante el cual un programa inicia múltiples flujos de control. Este proceso es esencial en la programación concurrente, ya que permite realizar múltiples tareas de manera simultánea dentro del mismo espacio de memoria. La gestión de estos hilos implica coordinar su ejecución, asignación de recursos, y la terminación de los mismos.

La creación de hilos varía según el lenguaje de programación y el sistema operativo. En muchos lenguajes de alto nivel como Java y Python, existen bibliotecas específicas para la gestión de hilos. Por ejemplo, en Java, la creación de un hilo se puede realizar mediante la implementación de la interfaz Runnable o extendiendo la clase Thread:


In [ ]:
class MyThread extends Thread {
    public void run() {
        System.out.println("Thread is running.");
    }
}

public class Main {
    public static void main(String[] args) {
        MyThread thread = new MyThread();
        thread.start();
    }
}


En Python, se utiliza el módulo threading para crear y gestionar hilos:

In [ ]:
import threading

def thread_function(name):
    print(f"Thread {name} is running.")

thread = threading.Thread(target=thread_function, args=(1,))
thread.start()
thread.join()


Por otro lado, en C se utiliza la biblioteca pthread para crear y manejar hilos en sistemas tipo Unix.

In [ ]:
#include <pthread.h>
#include <stdio.h>
#include <stdlib.h>

#define NUM_THREADS 5
#define NUM_ITERATIONS 1000000

// Contador global
int counter = 0;
// Mutex para sincronización
pthread_mutex_t mutex;

void* increment_counter(void* arg) {
    for (int i = 0; i < NUM_ITERATIONS; i++) {
        // Bloquear el mutex antes de acceder al contador
        pthread_mutex_lock(&mutex);
        counter++;
        // Desbloquear el mutex después de acceder al contador
        pthread_mutex_unlock(&mutex);
    }
    pthread_exit(NULL);
}

int main() {
    pthread_t threads[NUM_THREADS];
    int thread_ids[NUM_THREADS];
    int rc;

    // Inicializar el mutex
    pthread_mutex_init(&mutex, NULL);

    // Crear los hilos
    for (int i = 0; i < NUM_THREADS; i++) {
        thread_ids[i] = i;
        rc = pthread_create(&threads[i], NULL, increment_counter, (void*)&thread_ids[i]);
        if (rc) {
            printf("Error: No se pudo crear el hilo %d\n", i);
            exit(-1);
        }
    }

    // Esperar a que los hilos terminen
    for (int i = 0; i < NUM_THREADS; i++) {
        pthread_join(threads[i], NULL);
    }

    // Destruir el mutex
    pthread_mutex_destroy(&mutex);

    // Imprimir el valor final del contador
    printf("Valor final del contador: %d\n", counter);

    // Finalizar el programa
    pthread_exit(NULL);
}


La gestión de hilos incluye la creación, inicio, suspensión, reanudación y terminación de hilos. Además, se debe manejar la comunicación y la sincronización entre hilos para evitar condiciones de carrera y garantizar la coherencia de los datos compartidos.

**Sincronización de hilos**

La sincronización de hilos es crucial para coordinar las actividades de múltiples hilos que comparten recursos. Existen varias herramientas y técnicas para la sincronización de hilos, incluyendo mutexes, semáforos y barreras.

Mutexes

Un mutex (de MUTual EXclusion) es un mecanismo de sincronización que permite a los hilos controlar el acceso a recursos compartidos. Es fundamentalmente un bloqueo binario que puede estar en uno de dos estados: bloqueado o desbloqueado. Un hilo que desea acceder a un recurso protegido por un mutex intenta "adquirir" el mutex:

- Si el mutex está desbloqueado, el sistema operativo lo asigna al hilo solicitante, bloqueándolo automáticamente.
- Si el mutex ya está bloqueado por otro hilo, el hilo solicitante puede ser puesto en espera hasta que el mutex sea liberado (bloqueo) o puede realizar otra tarea hasta que intente adquirir el mutex nuevamente (spinlock).

Los mutexes son especialmente útiles en la programación multihilo para asegurar que solo un hilo acceda a un recurso compartido a la vez, previniendo así condiciones de carrera. Por ejemplo, al actualizar una variable compartida o al escribir en un archivo común, se utiliza un mutex para asegurar que las operaciones no se solapen de manera inapropiada.

In [ ]:
import threading

mutex = threading.Lock()

def critical_section():
    mutex.acquire()
    try:
        # Código de la sección crítica
        pass
    finally:
        mutex.release()


**Problemas potenciales**

A pesar de su utilidad, los mutexes pueden ser una fuente de errores difíciles de rastrear si no se manejan correctamente:

- Deadlocks: Si varios hilos intentan adquirir un conjunto de mutexes en un orden diferente, puede ocurrir un interbloqueo.
- Starvation (inanición): Un hilo puede quedar indefinidamente esperando para adquirir un mutex mientras otros hilos continúan adquiriéndolo y liberándolo.
- Prioridad inversa: En sistemas con prioridades de hilo, un hilo de baja prioridad que posee un mutex puede retrasar la ejecución de hilos de alta prioridad que esperan el mismo mutex.

Semáforos

Los semáforos son una herramienta fundamental en la programación multihilo, utilizados para controlar el acceso a recursos compartidos y coordinar la ejecución de hilos. Un semáforo es esencialmente una variable entera que, además de mantener su valor actual, ofrece dos operaciones atómicas principales: wait (también conocido como P o down) y signal (también conocido como V o up). Estas operaciones permiten a los semáforos gestionar el acceso a recursos de manera segura y eficiente.

Conceptos básicos de semáforos

- Valor del Semáforo: El valor de un semáforo indica la cantidad de recursos disponibles. Un valor inicial alto permite más accesos simultáneos a los recursos controlados por el semáforo.
- Operación wait (P o down): Esta operación decrementa el valor del semáforo. Si el valor resultante es negativo, el proceso que ejecuta la operación se bloquea hasta que el valor del semáforo sea positivo nuevamente.
- Operación signal (V o up): Esta operación incrementa el valor del semáforo. Si hay procesos bloqueados esperando a que el valor del semáforo sea positivo, uno de esos procesos es despertado.

Los semáforos pueden ser de dos tipos: binarios y contadores.

- Un semáforo binario funciona de manera similar a un mutex, permitiendo el acceso exclusivo a un recurso. El valor de un semáforo binario es 0 o 1.

In [ ]:
import threading

# Crear un semáforo binario
binary_semaphore = threading.Semaphore(1)

def critical_section():
    binary_semaphore.acquire()
    try:
        # Código de la sección crítica
        pass
    finally:
        binary_semaphore.release()


* Un semáforo contador permite que un número específico de hilos accedan a los recursos simultáneamente. Este tipo de semáforo es útil cuando se desea limitar el número de conexiones o accesos concurrentes a un recurso.

In [ ]:
import threading

# Crear un semáforo contador con un valor inicial de 3
counting_semaphore = threading.Semaphore(3)

def limited_resource_access():
    counting_semaphore.acquire()
    try:
        # Código que accede al recurso limitado
        pass
    finally:
        counting_semaphore.release()

**Ventajas y desventajas de los semáforos**

- Los semáforos pueden ser utilizados para diversos tipos de sincronización, desde exclusión mutua hasta gestión de recursos limitados.
- La implementación de semáforos es sencilla y clara, facilitando su comprensión y uso.
- Los semáforos permiten un acceso eficiente a recursos compartidos, minimizando el tiempo de espera de los hilos.
- En sistemas complejos, el uso de múltiples semáforos puede llevar a una lógica complicada y propensa a errores.
- Si no se gestionan correctamente, los semáforos pueden causar deadlocks, donde dos o más hilos quedan bloqueados indefinidamente.
- Los problemas de sincronización pueden ser difíciles de detectar y depurar, especialmente en programas grandes y concurrentes.

Barreras

Una barrera es un tipo de sincronización utilizado para hacer que un grupo de hilos espere hasta que todos los hilos hayan alcanzado un punto particular en su ejecución antes de que cualquiera de ellos pueda proceder. Este método es útil en procesos donde diferentes hilos deben realizar una parte del trabajo cada uno y sincronizarse en varios puntos para continuar.

La implementación de barreras generalmente implica una variable de contador y una variable de condición:

- Cada hilo que alcanza la barrera decrementa el contador.
- Si el contador no ha llegado a cero, el hilo se bloquea en una variable de condición.
- Cuando el último hilo llega y decrementa el contador a cero, señala la variable de condición para despertar a todos los hilos bloqueados.

Las barreras son útiles en aplicaciones que requieren que varios hilos de ejecución trabajen en pasos sincronizados. Un ejemplo podría ser el procesamiento de una imagen grande donde cada hilo maneja una sección de la imagen y todos deben sincronizarse después de completar cada paso, como la carga de la imagen, el procesamiento y la escritura del resultado.

Al igual que con otros mecanismos de sincronización, el uso incorrecto de barreras puede llevar a problemas de rendimiento, como la espera innecesaria si uno de los hilos tarda mucho más que los otros en alcanzar la barrera. Además, si no todos los hilos alcanzan la barrera (por ejemplo, debido a una terminación anticipada), puede resultar en un bloqueo permanente de los hilos que sí lo hicieron.

Las barreras son comúnmente utilizadas en algoritmos paralelos donde los hilos necesitan sincronizarse en ciertos puntos antes de proceder. Esto es útil en operaciones como el procesamiento en etapas, donde cada etapa depende del resultado de la anterior.

En aplicaciones que requieren cálculos en múltiples fases, las barreras aseguran que todas las partes del cálculo estén completas antes de pasar a la siguiente fase. Esto es crucial en tareas como simulaciones científicas y gráficos por computadora.

Ambos, mutexes y barreras, son herramientas esenciales en la caja de herramientas de un desarrollador de software para manejar la ejecución concurrente de manera eficiente y segura, facilitando la implementación de programas robustos y eficientes en entornos multihilo.

In [ ]:
import threading
import time
import random

# Crear una barrera para 3 hilos
barrier = threading.Barrier(3)

def worker(barrier, id):
    print(f'Thread {id} is starting.')
    time.sleep(random.uniform(0.1, 1.0))
    print(f'Thread {id} is waiting at the barrier.')
    barrier.wait()
    print(f'Thread {id} has passed the barrier.')

# Crear y lanzar hilos
threads = [threading.Thread(target=worker, args=(barrier, i)) for i in range(3)]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()


### Problemas clásicos en programación multihilo

La programación multihilo es una técnica poderosa que permite a las aplicaciones realizar múltiples tareas de manera simultánea y eficiente. Sin embargo, la gestión de múltiples hilos de ejecución simultáneamente introduce una serie de problemas complejos. Entre los más significativos se encuentran los deadlocks, livelocks y condiciones de carrera. Estos problemas no solo son comunes, sino que también pueden ser difíciles de detectar, diagnosticar y corregir, lo que requiere un entendimiento profundo de cómo los hilos interactúan entre sí y con los recursos compartidos.

**Deadlocks**

Un deadlock ocurre cuando dos o más hilos se bloquean indefinidamente, cada uno esperando que los otros liberen los recursos que están utilizando. Este tipo de situación se conoce como "interbloqueo" y puede ser visualizado como un círculo de hilos donde cada hilo sostiene al menos un recurso y espera adquirir otro que está siendo retenido por otro hilo en el círculo.

Causas comunes de deadlocks:

- Recursos no compartibles: Cuando un recurso no puede ser utilizado por más de un hilo al mismo tiempo sin causar conflictos o errores.
- Retención y espera: Un hilo que retiene recursos ya asignados mientras espera por recursos adicionales.
- No preemptividad: Los recursos que un hilo ha adquirido no pueden ser forzosamente removidos hasta que el hilo los libere.
- Espera circular: Cada hilo espera por un recurso que está siendo retenido por otro hilo, formando un ciclo de espera.

Estrategias de prevención y mitigación:

- Asignación de orden de recursos: Imponer un orden global en el que todos los hilos deben solicitar recursos, lo cual elimina la posibilidad de ciclos de espera.
- Recuperación de recursos: Diseñar mecanismos que permitan interrumpir y recuperar recursos de hilos que están en espera, aunque esto puede ser difícil o imposible en muchos contextos.
- Detección de Deadlocks: Implementar algoritmos que periódicamente verifiquen si existe algún ciclo de espera entre hilos y tomar medidas correctivas.

**Livelocks**

Un livelock es una condición en la que dos o más hilos continúan ejecutándose y respondiendo a las acciones de cada uno, sin embargo, ninguno de los hilos puede hacer un progreso real hacia la conclusión de su tarea. A diferencia de un deadlock, donde los hilos quedan completamente bloqueados, en un livelock los hilos están activamente cambiando su estado o respondiendo a condiciones, pero sin progreso productivo.

Causas comunes de livelocks:

- Respuesta excesiva a condiciones: Hilos que están diseñados para responder a ciertas condiciones de manera que sus respuestas provocan que otros hilos cambien sus estados, llevando a un ciclo continuo de cambios que impide el progreso.
- Estrategias de prevención de deadlocks ineficaces: Paradójicamente, algunas estrategias para evitar deadlocks, como liberar todos los recursos y volver a intentar, pueden llevar a livelocks si todos los hilos intentan hacerlo simultáneamente.

Estrategias de prevención y mitigación:

- Backoff aleatorio: Implementar un mecanismo de "retroceso" donde los hilos en conflicto esperan un tiempo aleatorio antes de reintentar la operación, lo que reduce la probabilidad de volver a entrar en el livelock.
- Priorización de hilos: Asignar diferentes prioridades a los hilos para que tengan más oportunidades de completar sus tareas sin interferir continuamente entre sí.

**Condiciones de carrera**

Las condiciones de carrera ocurren cuando el resultado de múltiples hilos depende del orden no determinístico en que se ejecutan las instrucciones. Este problema es especialmente peligroso porque puede introducir errores sutiles y difíciles de reproducir en el software.

Causas comunes de condiciones de carrera:

- Acceso concurrente a recursos compartidos: Varios hilos modifican o leen recursos compartidos sin los mecanismos adecuados para sincronización o exclusión mutua.
- Diseño de software inadecuado: Falta de comprensión o consideración durante el diseño del software sobre cómo las operaciones concurrentes afectarán el estado del programa.

Estrategias de prevención y mitigación:

* Bloqueos y sincronización: Utilizar mutexes, semáforos y otras primitivas de sincronización para controlar el acceso a recursos compartidos.
* Diseño de software consciente de la concurrencia: Diseñar sistemas con modelos claros de propiedad y transferencia de recursos, minimizando la necesidad de acceso concurrente.

Los problemas de programación multihilo requieren una atención meticulosa al detalle y un diseño cuidadoso para evitar estos problemas comunes, que pueden ser desafiantes pero son cruciales para la creación de aplicaciones robustas y eficientes.

#### Ejercicios

* Explica, utilizando tus propias palabras, ¿qué es un deadlock y bajo qué condiciones puede ocurrir en un sistema multihilo?
* Describe un escenario teórico en un sistema de manejo de base de datos donde los livelocks podrían ocurrir. ¿Cómo se podrían prevenir?
* ¿Por qué son problemáticas las condiciones de carrera y qué estrategias podrían utilizarse para evitarlas en el diseño de software?


In [ ]:
## Tus respuestas

1 . Simule un escenario simple de deadlock y luego resolverlo.

In [ ]:
import threading

# Recursos
lock1 = threading.Lock()
lock2 = threading.Lock()

def thread1():
    with lock1:
        print("Hilo 1: ha adquirido lock1")
        with lock2:
            print("Hilo 1: ha adquirido lock2")

def thread2():
    with lock2:
        print("Hilo 2: ha adquirido lock2")
        with lock1:
            print("Hilo 2: ha adquirido lock1")

t1 = threading.Thread(target=thread1)
t2 = threading.Thread(target=thread2)

t1.start()
t2.start()

t1.join()
t2.join()


Modifica el código para evitar el deadlock utilizando un orden de adquisición consistente de los locks.

In [ ]:
## Tu respuesta

2 .  Demostrar una condición de carrera y aplicar una solución.

In [ ]:
import threading

counter = 0

def increment():
    global counter
    for _ in range(100000):
        counter += 1

def decrement():
    global counter
    for _ in range(100000):
        counter -= 1

t1 = threading.Thread(target=increment)
t2 = threading.Thread(target=decrement)

t1.start()
t2.start()

t1.join()
t2.join()

print("Valor final del contador:", counter)


Implementa un mecanismo de bloqueo para evitar la condición de carrera.


In [ ]:
## Tu respuesta

3 . Crea un ejemplo de deadlock usando pthread.

In [ ]:
#include <pthread.h>
#include <stdio.h>

pthread_mutex_t lock1 = PTHREAD_MUTEX_INITIALIZER;
pthread_mutex_t lock2 = PTHREAD_MUTEX_INITIALIZER;

void* function1(void* data) {
    pthread_mutex_lock(&lock1);
    printf("Hilo 1: ha adquirido lock1\n");
    pthread_mutex_lock(&lock2);
    printf("Hilo 1: ha adquirido lock2\n");
    pthread_mutex_unlock(&lock2);
    pthread_mutex_unlock(&lock1);
    return NULL;
}

void* function2(void* data) {
    pthread_mutex_lock(&lock2);
    printf("Hilo 2: ha adquirido lock2\n");
    pthread_mutex_lock(&lock1);
    printf("Hilo 2: ha adquirido lock1\n");
    pthread_mutex_unlock(&lock1);
    pthread_mutex_unlock(&lock2);
    return NULL;
}

int main() {
    pthread_t thread1, thread2;
    pthread_create(&thread1, NULL, function1, NULL);
    pthread_create(&thread2, NULL, function2, NULL);
    pthread_join(thread1, NULL);
    pthread_join(thread2, NULL);
    return 0;
}


Modifica el orden de los locks para evitar el deadlock.

In [ ]:
## Tu respuesta

### Algoritmos de sincronización

Los algoritmos de sincronización son fundamentales para coordinar la ejecución de múltiples hilos en un programa, asegurando que se acceda de manera segura a los recursos compartidos y se respeten las restricciones de orden de ejecución. En el contexto de programación multihilo y sistemas distribuidos, los algoritmos de sincronización ayudan a resolver problemas como la exclusión mutua, la coordinación de tareas concurrentes y la prevención de condiciones de carrera, deadlocks y livelocks.

**Algoritmo de Lamport**

Desarrollado por Leslie Lamport, este algoritmo es fundamental para resolver problemas de sincronización en sistemas distribuidos sin un reloj global. Utiliza marcas de tiempo para asegurar la exclusión mutua y mantener un orden entre los eventos críticos. Cada proceso mantiene una marca de tiempo y, antes de entrar en una sección crítica, envía una solicitud con su marca de tiempo a todos los demás procesos y espera su aprobación.

**Algoritmo de Ricart-Agrawala**

Este es otro algoritmo basado en el paso de mensajes, que mejora el algoritmo de Lamport al reducir el número de mensajes necesarios para entrar en una sección crítica. Un proceso que desea entrar en una sección crítica envía una solicitud a todos los demás procesos y espera recibir un mensaje de aprobación de todos aquellos procesos cuyas solicitudes tienen una prioridad inferior.

**Algoritmos de productores y consumidores**
El problema de los productores y consumidores es un problema clásico de sincronización que involucra a dos tipos de procesos: productores y consumidores. Los productores generan datos y los colocan en un búfer compartido, mientras que los consumidores toman datos del búfer para procesarlos. El desafío radica en coordinar el acceso al búfer compartido para evitar condiciones de carrera y asegurar que ni los productores ni los consumidores accedan al búfer de manera conflictiva.

**Algoritmo de los filósofos comensales**

El problema de los filósofos comensales es un problema de sincronización clásico que describe la situación de cinco filósofos sentados alrededor de una mesa circular. Entre cada par de filósofos hay un tenedor. Para comer, un filósofo necesita tener ambos tenedores a su disposición, lo que puede llevar a situaciones de deadlock o livelock si no se gestiona adecuadamente.



Veamos algunas implementaciones base:

In [ ]:
## Codigo inicial del algoritmo de Lamport
import threading
import time
import random

class LamportClock:
    def __init__(self, pid):
        self.clock = 0
        self.pid = pid

    def increment(self):
        self.clock += 1

    def update(self, timestamp):
        self.clock = max(self.clock, timestamp) + 1

    def get_time(self):
        return self.clock

class Process:
    def __init__(self, pid, total_processes):
        self.pid = pid
        self.clock = LamportClock(pid)
        self.total_processes = total_processes
        self.request_queue = []

    def request_resource(self):
        self.clock.increment()
        timestamp = self.clock.get_time()
        request = (timestamp, self.pid)
        self.request_queue.append(request)
        self.request_queue.sort()
        return request

    def receive_request(self, timestamp, pid):
        self.clock.update(timestamp)
        self.request_queue.append((timestamp, pid))
        self.request_queue.sort()

    def release_resource(self):
        self.request_queue.pop(0)

    def can_enter_critical_section(self):
        if self.request_queue:
            return self.request_queue[0][1] == self.pid
        return False

def simulate_process(process):
    while True:
        time.sleep(random.uniform(0.5, 2))
        request = process.request_resource()
        print(f"Process {process.pid} requested resource at time {request[0]}")

        # Simulate receiving requests from other processes
        for i in range(process.total_processes):
            if i != process.pid:
                process.receive_request(random.randint(0, 10), i)

        while not process.can_enter_critical_section():
            time.sleep(0.1)
        
        print(f"Process {process.pid} entering critical section at time {process.clock.get_time()}")
        time.sleep(random.uniform(0.5, 1.5))
        print(f"Process {process.pid} leaving critical section at time {process.clock.get_time()}")
        process.release_resource()

total_processes = 3
processes = [Process(i, total_processes) for i in range(total_processes)]
threads = [threading.Thread(target=simulate_process, args=(p,)) for p in processes]

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()


In [ ]:
## Codigo inicial del algoritmo de Ricart-Agrawala
import threading
import time
import random

class RicartAgrawala:
    def __init__(self, pid, total_processes):
        self.pid = pid
        self.clock = 0
        self.total_processes = total_processes
        self.request_queue = []
        self.deferred_replies = []
        self.requesting_cs = False
        self.lock = threading.Lock()

    def request_resource(self):
        with self.lock:
            self.clock += 1
            self.requesting_cs = True
            request = (self.clock, self.pid)
            self.request_queue.append(request)
            self.request_queue.sort()
            return request

    def receive_request(self, timestamp, pid):
        with self.lock:
            self.clock = max(self.clock, timestamp) + 1
            request = (timestamp, pid)
            self.request_queue.append(request)
            self.request_queue.sort()
            if self.requesting_cs and (timestamp, pid) < (self.clock, self.pid):
                self.deferred_replies.append(pid)
            else:
                return True
        return False

    def receive_reply(self, pid):
        with self.lock:
            self.deferred_replies.remove(pid)

    def release_resource(self):
        with self.lock:
            self.request_queue.pop(0)
            self.requesting_cs = False
            replies = self.deferred_replies[:]
            self.deferred_replies.clear()
        return replies

    def can_enter_critical_section(self):
        with self.lock:
            if self.request_queue:
                return self.request_queue[0][1] == self.pid
            return False

def simulate_process(process):
    while True:
        time.sleep(random.uniform(0.5, 2))
        request = process.request_resource()
        print(f"Process {process.pid} requested resource at time {request[0]}")

        # Simulate receiving requests from other processes
        for i in range(process.total_processes):
            if i != process.pid:
                process.receive_request(random.randint(0, 10), i)

        while not process.can_enter_critical_section():
            time.sleep(0.1)
        
        print(f"Process {process.pid} entering critical section at time {process.clock}")
        time.sleep(random.uniform(0.5, 1.5))
        print(f"Process {process.pid} leaving critical section at time {process.clock}")
        deferred_replies = process.release_resource()

        # Simulate sending replies to deferred processes
        for pid in deferred_replies:
            print(f"Process {process.pid} sending reply to process {pid}")

total_processes = 3
processes = [RicartAgrawala(i, total_processes) for i in range(total_processes)]
threads = [threading.Thread(target=simulate_process, args=(p,)) for p in processes]

for thread in threads:
    thread.start()
for thread in threads:
    thread.join()


In [ ]:
## Algoritmo de productores y consumidores
import threading
import time
import random

buffer = []
buffer_size = 10

empty = threading.Semaphore(buffer_size)
full = threading.Semaphore(0)
mutex = threading.Semaphore(1)

def producer():
    while True:
        item = random.randint(1, 100)
        empty.acquire()
        mutex.acquire()
        buffer.append(item)
        print(f'Producer produced {item}')
        mutex.release()
        full.release()
        time.sleep(random.random())

def consumer():
    while True:
        full.acquire()
        mutex.acquire()
        item = buffer.pop(0)
        print(f'Consumer consumed {item}')
        mutex.release()
        empty.release()
        time.sleep(random.random())

producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)
producer_thread.start()
consumer_thread.start()
producer_thread.join()
consumer_thread.join()


In [ ]:
## Problema de los filósofos comensales
import threading
import time
import random

N = 5
forks = [threading.Semaphore(1) for _ in range(N)]

def philosopher(id):
    left_fork = id
    right_fork = (id + 1) % N

    while True:
        print(f'Philosopher {id} is thinking.')
        time.sleep(random.random())

        forks[left_fork].acquire()
        print(f'Philosopher {id} picked up left fork {left_fork}.')
        forks[right_fork].acquire()
        print(f'Philosopher {id} picked up right fork {right_fork}.')

        print(f'Philosopher {id} is eating.')
        time.sleep(random.random())

        forks[right_fork].release()
        print(f'Philosopher {id} put down right fork {right_fork}.')
        forks[left_fork].release()
        print(f'Philosopher {id} put down left fork {left_fork}.')

philosophers = [threading.Thread(target=philosopher, args=(i,)) for i in range(N)]
for philosopher_thread in philosophers:
    philosopher_thread.start()
for philosopher_thread in philosophers:
    philosopher_thread.join()


#### Ejercicios

* Crea un programa que lance cinco hilos. Cada hilo debe imprimir diez veces su identificador único y luego terminar.
* Desarrolla un programa que cree tres hilos. Cada hilo debe incrementar un contador compartido 100 veces. Implementa una forma de esperar a que todos los hilos terminen antes de imprimir el valor final del contador.
* Implementa un programa que use semáforos para controlar el acceso a un recurso compartido por tres hilos.
* Utiliza mutexes para proteger la escritura a una variable compartida en un entorno multihilo. Asegúrate de que cada hilo pueda actualizar la variable de forma segura.
* Explica la diferencia entre un semáforo binario y un mutex. ¿En qué situaciones es preferible usar cada uno?
*  Describe un escenario en el que el uso de una barrera sería crítico en un sistema de procesamiento de datos distribuido. ¿Cómo asegurarías que todos los hilos alcancen la barrera de manera eficiente sin desperdiciar recursos del CPU?
* Propón un algoritmo de detección de deadlock para un sistema con múltiples tipos de recursos. Considera cómo podrías implementar este algoritmo en un sistema operativo real.
* Describe un escenario donde un sistema de intentos de recuperación automática podría llevar a un livelock. Propón una solución basada en el diseño del software.
* 
* Crea un escenario de deadlock utilizando dos mutexes y dos hilos. Cada hilo intenta adquirir los mutexes en orden diferente.
* Desarrolla un ejemplo de condición de carrera donde dos hilos incrementan y decrementan, respectivamente, un contador compartido sin sincronización adecuada.
* Considera un sistema de votación online donde se esperan múltiples accesos concurrentes. Describe cómo podrías diseñar el sistema para evitar condiciones de carrera sin degradar el rendimiento.
* Describe cómo el algoritmo de Lamport para la exclusión mutua puede ser aplicado en un sistema de reserva de entradas en línea. ¿Qué desafíos podrían surgir y cómo los resolverías?
*  Analiza el algoritmo de Ricart-Agrawala en el contexto de un sistema de control de tráfico aéreo. ¿Cómo se manejarían las prioridades de los mensajes?
* Implementa el algoritmo del productor-consumidor usando colas bloqueantes y semáforos.
* Escribe un programa que simule el problema de los filósofos comensales utilizando semáforos para evitar deadlocks y asegurar que todos los filósofos puedan comer sin interferir entre ellos.

In [ ]:
## Tus respuestas

1 . Implementa un sistema simple de productor-consumidor usando semáforos y mutexes.

In [ ]:
import threading
import time
import random
from collections import deque

buffer = deque()
buffer_size = 5
semaphore = threading.Semaphore(0)
mutex = threading.Lock()

def producer():
    while True:
        item = random.randint(1, 100)
        time.sleep(random.random())
        mutex.acquire()
        if len(buffer) < buffer_size:
            buffer.append(item)
            print(f"Productor produjo: {item}")
            semaphore.release()
        mutex.release()

def consumer():
    while True:
        semaphore.acquire()
        mutex.acquire()
        item = buffer.popleft()
        print(f"Consumidor consumió: {item}")
        mutex.release()
        time.sleep(random.random())

producer_thread = threading.Thread(target=producer)
consumer_thread = threading.Thread(target=consumer)

producer_thread.start()
consumer_thread.start()


2 . Utiliza barreras para sincronizar hilos en un proceso de tres etapas.

In [ ]:
import threading

barrier = threading.Barrier(3)

def stage(worker_id):
    print(f"Hilo {worker_id} esperando en la barrera.")
    barrier.wait()
    print(f"Hilo {worker_id} pasó la barrera.")

threads = [threading.Thread(target=stage, args=(i,)) for i in range(3)]
for t in threads:
    t.start()
for t in threads:
    t.join()


3 . Crea y resuelve un deadlock y simula un livelock.

In [ ]:
# Deadlock
lock1 = threading.Lock()
lock2 = threading.Lock()

def deadlock_thread_1():
    with lock1:
        time.sleep(0.1)
        with lock2:
            print("Hilo 1 completado")

def deadlock_thread_2():
    with lock2:
        time.sleep(0.1)
        with lock1:
            print("Hilo 2 completado")

# Livelock
def livelock_thread_1():
    while True:
        with lock1:
            if not lock2.locked():
                print("Livelock evitado por Hilo 1")
                break
            print("Hilo 1 reintentando...")
            time.sleep(0.1)

def livelock_thread_2():
    while True:
        with lock2:
            if not lock1.locked():
                print("Livelock evitado por Hilo 2")
                break
            print("Hilo 2 reintentando...")
            time.sleep(0.1)

# Ejecución
thread1 = threading.Thread(target=deadlock_thread_1)
thread2 = threading.Thread(target=deadlock_thread_2)
thread1.start()
thread2.start()
thread1.join()
thread2.join()


4 . Demuestra y soluciona una condición de carrera.

In [ ]:
counter = 0
lock = threading.Lock()

def unsafe_update():
    global counter
    local_copy = counter
    local_copy += 1
    time.sleep(0.1)  # Simular carga de trabajo
    counter = local_copy

def safe_update():
    global counter
    with lock:
        local_copy = counter
        local_copy += 1
        time.sleep(0.1)  # Simular carga de trabajo
        counter = local_copy

# Cambiar safe_update por unsafe_update para ver el efecto de una condición de carrera
update_thread1 = threading.Thread(target=safe_update)
update_thread2 = threading.Thread(target=safe_update)
update_thread1.start()
update_thread2.start()
update_thread1.join()
update_thread2.join()
print("Valor final del contador:", counter)


5 .  Implementa una solución al problema de los filósofos comensales para evitar deadlocks y starvation.

In [ ]:
def philosopher(left_fork, right_fork, philosopher_number):
    while True:
        with left_fork:
            with right_fork:
                print(f"Filósofo {philosopher_number} está comiendo.")
                time.sleep(0.5)
        print(f"Filósofo {philosopher_number} está pensando.")

forks = [threading.Lock() for _ in range(5)]
philosophers = [threading.Thread(target=philosopher, args=(forks[i % 5], forks[(i + 1) % 5], i)) for i in range(5)]
for p in philosophers:
    p.start()


6 . Explica cómo los semáforos pueden ayudar a resolver el problema del productor-consumidor. Compare el uso de semáforos con otras técnicas de sincronización como los mutexes y las barreras en este contexto.

In [ ]:
import threading
import random
import time
from collections import deque

buffer = deque()
buffer_size = 10
empty = threading.Semaphore(buffer_size)
full = threading.Semaphore(0)
mutex = threading.Lock()

def producer():
    while True:
        item = random.randint(1, 100)
        empty.acquire()
        mutex.acquire()
        buffer.append(item)
        print(f"Producer produced: {item}")
        mutex.release()
        full.release()
        time.sleep(random.uniform(0.1, 1))

def consumer():
    while True:
        full.acquire()
        mutex.acquire()
        item = buffer.popleft()
        print(f"Consumer consumed: {item}")
        mutex.release()
        empty.release()
        time.sleep(random.uniform(0.1, 1))

producers = [threading.Thread(target=producer) for _ in range(3)]
consumers = [threading.Thread(target=consumer) for _ in range(3)]

for p in producers:
    p.start()
for c in consumers:
    c.start()


7 . Explica el concepto de "inversión de prioridad" en sistemas de mutexes. Propón una solución para mitigar este problema en un sistema de control de acceso concurrente.

In [ ]:
import threading

counter = 0
counter_mutex = threading.Lock()

def increment():
    global counter
    for _ in range(100000):
        with counter_mutex:
            counter += 1

def decrement():
    global counter
    for _ in range(100000):
        with counter_mutex:
            counter -= 1

t1 = threading.Thread(target=increment)
t2 = threading.Thread(target=decrement)

t1.start()
t2.start()

t1.join()
t2.join()

print("Final counter value:", counter)


8 . Explica la diferencia entre deadlock y livelock. Propón un método para detectar y resolver livelocks en un sistema distribuido.


In [ ]:
import threading
import time

lock1 = threading.Lock()
lock2 = threading.Lock()

def livelock_thread_1():
    while True:
        with lock1:
            if not lock2.locked():
                with lock2:
                    print("Thread 1 acquired both locks")
                    return
        print("Thread 1 retrying")
        time.sleep(0.1)

def livelock_thread_2():
    while True:
        with lock2:
            if not lock1.locked():
                with lock1:
                    print("Thread 2 acquired both locks")
                    return
        print("Thread 2 retrying")
        time.sleep(0.1)

t1 = threading.Thread(target=livelock_thread_1)
t2 = threading.Thread(target=livelock_thread_2)

t1.start()
t2.start()

t1.join()
t2.join()


9 . Describe cómo funciona el algoritmo de Ricart-Agrawala para la exclusión mutua en sistemas distribuidos. Compara este algoritmo con el de Lamport en términos de eficiencia y complejidad.

In [ ]:
import threading
import queue

ricart_agrawala_queue = queue.Queue()

def request_critical_section(node_id):
    global lamport_clock
    lamport_clock += 1
    ricart_agrawala_queue.put((lamport_clock, node_id))
    print(f"Node {node_id} requested critical section with timestamp: {lamport_clock}")

def receive_request():
    while True:
        if not ricart_agrawala_queue.empty():
            timestamp, node_id = ricart_agrawala_queue.get()
            print(f"Received request from Node {node_id} with timestamp: {timestamp}")

t1 = threading.Thread(target=request_critical_section, args=(1,))
t2 = threading.Thread(target=receive_request)

t1.start()
t2.start()

t1.join()
t2.join()


10 . Explica el problema de los filósofos comensales y discuta las posibles soluciones para evitar deadlocks y condiciones de inanición.

In [ ]:
import threading
import time
import random

N = 5
chopsticks = [threading.Lock() for _ in range(N)]

def philosopher(id):
    left = id
    right = (id + 1) % N
    while True:
        time.sleep(random.uniform(0.1, 1))
        print(f"Philosopher {id} is thinking.")
        with chopsticks[left]:
            with chopsticks[right]:
                print(f"Philosopher {id} is eating.")
                time.sleep(random.uniform(0.1, 1))

threads = [threading.Thread(target=philosopher, args=(i,)) for i in range(N)]
for t in threads:
    t.start()

for t in threads:
    t.join()


In [ ]:
## Tus respuestas